## Problem 1
Consider the network flow in the diagram below. Each circle contains a number representing a location. Each diamond shape contains a number representing the supply or demand for a given commodity at each location. Each octagon shape contains a number representing the cost (per unit) for transporting the commodity from one location to another. Each arrows represent the direction that the commodity can flow from a location to another.

Each row represents each path, and column c_ij represents the cost per unit on corresponding path:

| Start_node_i | End_node_j | c_ij |
| ----- | ------ | ----- | 
|  1    |	2	 |	2	 | 
|  1    |	3	 |	-5 | 
|  2    |	3	 |	-1 | 
|  2    |	4	 |	4 | 
|  3    |	4	 |	3 | 
|  3    |	2	 |	6 |
|  4    |	1	 |	7 |

#### Define our variables
There are 7 paths, so we have 7 variables:

* $x_{12}$ is the number of units that transported from location 1 to 2.
* $x_{13}$ is the number of units that transported from location 1 to 3.
* $x_{23}$ is the number of units that transported from location 2 to 3.
* $x_{24}$ is the number of units that transported from location 2 to 4.
* $x_{34}$ is the number of units that transported from location 3 to 4.
* $x_{32}$ is the number of units that transported from location 3 to 2.
* $x_{41}$ is the number of units that transported from location 4 to 1.

#### Regarding constraints:

For each location, the total units transported from the location minus the total units transported to the location must be less than the inventory level:

$$ x12 + x13 - x41 \leq 4 $$
$$ x23 + x24 - x12 \leq 2 $$
$$ x34 + x32 -x23 \leq -1 $$
$$ x41 + x24 -x34 \leq -5 $$



#### The objective function is defined as:

$$ min\ \  2x_{12} - 5x_{13} -1 x_{23} + 4x_{24} + 3x_{34} + 6x_{32} + 7x_{41} $$




In [1]:
using JuMP, GLPK

myModel = Model(GLPK.Optimizer)
@variable(myModel, x12>= 0)
@variable(myModel, x13>= 0)
@variable(myModel, x23>= 0)
@variable(myModel, x24>= 0)
@variable(myModel, x34>= 0)
@variable(myModel, x32>= 0)
@variable(myModel, x41>= 0)
@constraint(myModel, x12+x13-x41<=4)
@constraint(myModel, x23+x24-x12<=2)
@constraint(myModel, x34+x32-x23<=-1)
@constraint(myModel, x41-x24-x34<=-5)


@objective(myModel, Min, 2*x12-5*x13-1*x23+4*x24+3*x34+6*x32+7*x41)
myModel



A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x12, x13, x23, x24, x32, x34, x41

In [9]:
@time begin
    status = optimize!(myModel)
end
println("Objective value: ", JuMP.objective_value(myModel))
println("x12 = ", JuMP.value(x12))
println("x13 = ", JuMP.value(x13))
println("x23 = ", JuMP.value(x23))
println("x24 = ", JuMP.value(x24))
println("x34 = ", JuMP.value(x34))
println("x32 = ", JuMP.value(x32))
println("x41 = ", JuMP.value(x41))

  0.000022 seconds (2 allocations: 32 bytes)
Objective value: 17.0
x12 = 4.0
x13 = 0.0
x23 = 6.0
x24 = 0.0
x34 = 5.0
x32 = 0.0
x41 = 0.0


#### Optimal solution: 
Transport 4 units from location 1 to 2, transport 6 units from location 2 to 3, and  transport 5 units from location 3 to 4.  In this way, the total cost is 17.